In [ ]:
import pandas as pd
import numpy as np

origin_df = pd.read_csv('./04_hashed.csv')
origin_df

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler

# IPv4 전처리
def transform_ip(ip): 
    groups = ip.split(".") 
    equalize_group_length = "".join( map( lambda group: group.zfill(3), groups )) 
    return equalize_group_length 

# Feature 전처리
def preprocess_df(df):
      
    # IPv4 전처리
    df['src_ip'] = df.src_ip.apply(lambda ip : transform_ip(ip))
    df['dst_ip'] = df.dst_ip.apply(lambda ip : transform_ip(ip))

    # country 전처리
    label_encoder = LabelEncoder()
    df['src_country'] = label_encoder.fit_transform(df['src_country'])
    df['dst_country'] = label_encoder.fit_transform(df['dst_country'])

    # scaler 적용전 DataFrame (IP 와 country 만 전처리한 상태)
    df_without_scaler = df

    # kmeans, dbscan 등은 거리기반 알고리즘이기 때문에 단위의 영향력을 제거하기 위해 표준화 진행
    # (x-mean)/std 로 변환하여 평균 0, 표준편차 1로 scaling 을 맞춰주는 standardScaler 를 사용
    
    Robust_scaler = RobustScaler()
    # scaler 를 적용한 DataFrame
    feature_trans = Robust_scaler.fit_transform(df)
    df = pd.DataFrame(feature_trans, columns=df.columns, index=df.index)
    
    return df

In [ ]:
from sklearn.cluster import DBSCAN

def dbscan(input_df, min_samples, eps):
      
    feature_preprocess_df = input_df[["src_ip", "dst_ip", "Proto", "src_port", "dst_port", "Action", "src_country", "dst_country"]]
    # 하이퍼파라미터 변경 
    model = DBSCAN(min_samples=min_samples, eps=eps)
    
    y_label = model.fit_predict(feature_preprocess_df)
    # 모델 학습 및 예측 결과 생성 후, DataFrame 으로 변환
    y_pred = pd.DataFrame(y_label, columns=['y_pred'], index=feature_preprocess_df.index)
        
    # 입력 feature 와 target 을 concat
    result_df = pd.concat([feature_preprocess_df,y_pred],axis=1)

    # 군집화 결과
    labels = model.labels_

    # 이상치 군집을 제외한 군집화 종류 개수
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    
    # 이상치 개수
    n_noise_ = list(labels).count(-1)

    print("Dataset Size : ",len(feature_preprocess_df))
    print("군집화 전체 결과 : ", labels)
    print("이상치 군집을 제외한 군집화 종류 개수 : %d" % n_clusters_)
    print("이상치 개수 : %d" % n_noise_)
    return result_df, labels, feature_preprocess_df, y_label

In [ ]:
# Rdate 종류
unique_time = origin_df['Rdate'].unique()
len(unique_time)

scaled_df = origin_df[["src_ip", "dst_ip", "Proto", "src_port", "dst_port", "Action", "src_country", "dst_country"]]
scaled_df = preprocess_df(scaled_df.astype(str))
scaled_df['Rdate'] = origin_df['Rdate']

print(scaled_df)

In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
# window size 종류 만큼의 데이터 추출
window_size = 100

min_samples = 20
eps = 9
# 전체 데이터에 대한 이상치 군집화 결과
total_anomaly_df = pd.DataFrame()

# 전체 Rate 종류 탐색 시, 마지막 탐색 index 는 window_size 를 고려해 설정
last_search_index = len(unique_time)-window_size

count = 0



for i in range(0,last_search_index,window_size):
    
    # 임시 window df 생성
    window_df = pd.DataFrame()

    # window size 종류 만큼의 Rdate 만 확인
    for j in range(window_size):
        
        # 전체 데이터를 순서대로 확인하며, Rdate 종류에 해당하는 데이터를 추출해 window df 로 생성
        select_time = scaled_df[scaled_df['Rdate']==unique_time[i+j]]
        
        # y 축으로 concat
        window_df = pd.concat([window_df,select_time])
        
    # 추출된 window size 종류 만큼의 데이터만 가지고 DBSCAN 진행
    dbscan_result_df, labels, feature_trans, y_label = dbscan(window_df, min_samples, eps)
    
    # 이상치로 군집화된 데이터만 추출
    anomaly_df = dbscan_result_df[dbscan_result_df['y_pred']==-1]
    
    # 전체 데이터에 대한 이상치 군집화 결과에 추가
    total_anomaly_df = pd.concat([total_anomaly_df, anomaly_df])
    
    scatter_matrix(feature_trans,
               c = y_label,
               alpha=0.5,
               figsize=(20, 20),
               diagonal='kde')
    
    plt.show()
    
    count += window_size
    print(" @@@@@@@@@@@@@@@@@@@@ %d 개 Rdate 중, %d 개 확인 완료 @@@@@@@@@@@@@@@@@@@@ " % (len(unique_time), count))
    

In [ ]:
# 이상치 결과만 합쳐놓은 DataFrame
total_anomaly_df

In [ ]:
# total_anomaly_df 의 index 추출
anomaly_index = total_anomaly_df.index

# 정상은 1, 이상치는 -1 
target = []

for i in range(len(origin_df)):
    
    # 원본 DataFrame 를 오름차순으로 탐색하며
    # 해당 index 가 total_anomaly_df 에 있는 index 라면 Target 을 -1 로 설정
    if (i in anomaly_index):
        target.append(-1)
    else:
        target.append(1)        

In [ ]:
# target 내 이상치 개수
print(list(target).count(-1))
# target 내 정상 개수
print(list(target).count(1))
# 정상 + 이상
print(list(target).count(-1) + list(target).count(1))

In [ ]:
# 완성된 Target list 를 원본 DataFrame 에 Concat
result_df = origin_df
result_df['y_pred'] = target
result_df.to_csv("./unho_dbscan_result.csv", index=False)
result_df

In [ ]:
# import matplotlib.pyplot as plt
# from pandas.plotting import scatter_matrix

# # feature의 target 별로 색을 다르게 표현
# scatter_matrix(feature_preprocess_df,
#                c = y_pred,
#                alpha=0.5, 
#                figsize=(20, 20), 
#                diagonal='kde')

# plt.show()